In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing






In [8]:
!pip install h5py

In [9]:
import h5py
import numpy as np


def import_data(every=False):
    if every:
        electrodes = 25
    else:
        electrodes = 22
    X, y = [], []
    for i in range(9):
        A01T = h5py.File('/content/drive/My Drive/data/A0' + str(i + 1) + 'T_slice.mat', 'r')
        X1 = np.copy(A01T['image'])
        X.append(X1[:, :electrodes, :])
        y1 = np.copy(A01T['type'])
        y1 = y1[0, 0:X1.shape[0]:1]
        y.append(np.asarray(y1, dtype=np.int32))

    for subject in range(9):
        delete_list = []
        for trial in range(288):
            if np.isnan(X[subject][trial, :, :]).sum() > 0:
                delete_list.append(trial)
        X[subject] = np.delete(X[subject], delete_list, 0)
        y[subject] = np.delete(y[subject], delete_list)
    y = [y[i] - np.min(y[i]) for i in range(len(y))]
    return X, y


def train_test_subject(X, y, train_all=True, standardize=True):

    l = np.random.permutation(len(X[0]))
    X_test = X[0][l[:50], :, :]
    y_test = y[0][l[:50]]

    if train_all:
        X_train = np.concatenate((X[0][l[50:], :, :], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8]))
        y_train = np.concatenate((y[0][l[50:]], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8]))

    else:
        X_train = X[0][l[50:], :, :]
        y_train = y[0][l[50:]]

    X_train_mean = X_train.mean(0)
    X_train_var = np.sqrt(X_train.var(0))

    if standardize:
        X_train -= X_train_mean
        X_train /= X_train_var
        X_test -= X_train_mean
        X_test /= X_train_var

    X_train = np.transpose(X_train, (0, 2, 1))
    X_test = np.transpose(X_test, (0, 2, 1))

    return X_train, X_test, y_train, y_test


def train_test_total(X, y, standardize=True):

    X_total = np.concatenate((X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8]))
    y_total = np.concatenate((y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8]))

    l = np.random.permutation(len(X_total))
    X_test = X_total[l[:50], :, :]
    y_test = y_total[l[:50]]
    X_train = X_total[l[50:], :, :]
    y_train = y_total[l[50:]]

    X_train_mean = X_train.mean(0)
    X_train_var = np.sqrt(X_train.var(0))

    if standardize:
        X_train -= X_train_mean
        X_train /= X_train_var
        X_test -= X_train_mean
        X_test /= X_train_var

    X_train = np.transpose(X_train, (0, 2, 1))
    X_test = np.transpose(X_test, (0, 2, 1))

    return X_train, X_test, y_train, y_test


# Bidirectional LSTM

In [ ]:
# %%
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

# %%
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

# %%
def rnn_model(features, labels, mode, params):


  num_hidden_layers = len(params['hidden_layers'])

  outputs = features["x"]

  model = params['model']

  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):


      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)

      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  print(type(outputs))
  print(outputs.shape)
  #FLatten the output of LSTM layers
  outputs = tf.keras.layers.Flatten()(outputs)

  print(type(outputs))
  print(outputs.shape)
  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




# %%
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [256, 256], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.BasicRNNCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.GRUCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%





Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 64000)


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


Using activation -  <function tanh at 0x7f44c9f75990>


Instructions for updating:
Use tf.keras instead.


...

{'accuracy': 0.7535885, 'loss': 0.6606881, 'global_step': 100}
{'accuracy': 0.54, 'loss': 0.9481491, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)


{'accuracy': 0.75239235, 'loss': 0.6225511, 'global_step': 100}
{'accuracy': 0.66, 'loss': 0.812376, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)


{'accuracy': 0.761563, 'loss': 0.6178331, 'global_step': 100}
{'accuracy': 0.6, 'loss': 0.9036742, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


{'accuracy': 0.66586924, 'loss': 1.1482065, 'global_step': 100}
{'accuracy': 0.38, 'loss': 2.2814727, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
{'accuracy': 0.75, 'loss': 0.6398369, 'global_step': 100}
{'accuracy': 0.56, 'loss': 1.0955027, 'global_step': 100}
